In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tqdm import tqdm

from random import randrange

from datetime import datetime, timedelta, date

import re

import pandas as pd

import os

In [3]:
dialogues_dir = 'dialogues'
os.listdir(dialogues_dir)

['Oleg_Litvinov.csv',
 'Oleg_Litvinov.txt',
 'Влад Михайлович_Власенко.csv',
 'Влад_Михайлович_Власенко.txt',
 'Катя_Тимуровна.csv',
 'Полина.csv']

In [4]:
my_name = 'Oleg Litvinov'
companion_name = 'Катя_Тимуровна'

In [5]:
df = pd.read_csv(os.path.join(dialogues_dir, companion_name+'.csv'))
df.datetime = pd.to_datetime(df.datetime)
df.sort_values(['datetime'], inplace=True)
df.head(2)

,datetime,sender_id,sender_name,message
0,2017-03-20 22:34:59+00:00,47683,Oleg Litvinov,1. Берёшь крутой стикер\n2. Пишешь сообщение\n...
1,2017-03-20 22:35:10+00:00,47684,Oleg Litvinov,<MessageMediaDocument>


In [6]:
df = df[~df.isna().any(axis=1)]
df.shape

(252743, 4)

In [7]:
# df[df.message.apply(lambda x: re.search(r'(MessageMediaPhoto)', x) is not None)].message.apply(lambda x: re.sub(r'(<\w*Media\w*> )', '', x))

In [8]:
df.message = df.message.apply(lambda x: re.sub(r'(<\w*Media\w*> )', '', x))

df = df[df.message.apply(lambda x: len(x)) > 0]

df.shape

(238742, 4)

In [9]:
# df['from_prev_msg'] = df.datetime - df.datetime.shift(1)

# df.from_prev_msg = df.from_prev_msg.astype('timedelta64[h]')
# df

# df['new_topic'] = df['from_prev_msg'] > 4
# df.head()

In [10]:
df.reset_index(drop=True, inplace=True)
df.head(2)

,datetime,sender_id,sender_name,message
0,2017-03-20 22:34:59+00:00,47683,Oleg Litvinov,1. Берёшь крутой стикер\n2. Пишешь сообщение\n...
1,2017-03-20 22:35:14+00:00,47685,Oleg Litvinov,Привет


In [11]:
df.sender_name.value_counts()

Oleg Litvinov     124726
Катя Тимуровна    114016
Name: sender_name, dtype: int64

In [12]:
df['is_answer'] = df.sender_name != df.sender_name.shift(1)

In [56]:
def random_chunk(df, min_chunk=1, max_chunk=3):
    from random import randint
    df = df.copy()
    df.reset_index(drop=True, inplace=True)
    
    split_list = []
    L = len(df)
    i = 0
    pbar = tqdm(total=L)
    while i < L:
        r = randint(min(L - i, min_chunk), min(L - i, max_chunk))
        yield df.loc[i:i + r - 1, :] # including
        i = i + r
        pbar.update(r)
    pbar.close()

In [72]:
def generate_train_file(df):
    final_string = []
    
    for chunk in random_chunk(df, min_chunk=3, max_chunk=15):
        output_string = "<s>" + "\n".join((chunk['sender_name'] + ": " + chunk['message']).values) + "</s>"
#         print(output_string)
        final_string.append(output_string)
    
#     for index, series in tqdm(df.iterrows(), total=len(df)):
#         print(index)
#         if (series['is_answer'] == True) and (index > 10):
#             story_len = randrange(1, min(15, index))
#             print('story_len', story_len)
#             history_df = df.loc[index-story_len:index+1, ['sender_name', 'message']]
#             display(history_df)
#             output_string = "<s>" + "\n".join((history_df['sender_name'] + ": " + history_df['message']).values) + "</s>"
#             print(output_string)
#             final_string.append(output_string)
    lens = np.vectorize(len)(final_string)
    print(f"symbols min:{np.min(lens)}, mean:{np.mean(lens)}, max:{np.max(lens)}")
    
    final_string = "\n".join(final_string)
    return final_string

In [58]:
train_end = date(2020, 1, 1)
val_end = date(2021, 1, 1)
test_end = df.datetime.max().date() - timedelta(weeks=1)

In [59]:
train_df = df[df.datetime.dt.date < train_end]
val_df = df[(df.datetime.dt.date < val_end) & (df.datetime.dt.date >= train_end)]
test_df = df[(df.datetime.dt.date < test_end) & (df.datetime.dt.date >= val_end)]

In [60]:
train_df.shape, val_df.shape, test_df.shape

((173740, 5), (55438, 5), (9152, 5))

In [73]:
test_file = generate_train_file(test_df)

100%|███████████████████████████████████████████████████████████████████████████| 9152/9152 [00:00<00:00, 14628.17it/s]

symbols min:61, mean:382.90416263310743, max:2616


In [74]:
val_file = generate_train_file(val_df)

100%|█████████████████████████████████████████████████████████████████████████| 55438/55438 [00:04<00:00, 13605.53it/s]


symbols min:62, mean:422.65509868421054, max:5862


In [75]:
train_file = generate_train_file(train_df)

100%|███████████████████████████████████████████████████████████████████████| 173740/173740 [00:14<00:00, 12094.02it/s]


symbols min:62, mean:445.2875469336671, max:3043


In [64]:
def check_path(filepath):
    if not os.path.exists(os.path.dirname(filepath)):
        try:
            os.makedirs(os.path.dirname(filepath))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
            
def save_to_txt(file, path):
    check_path(path)
    with open(path, "w", encoding="utf-8") as text_file:
        text_file.write(file)

In [66]:
save_to_txt(train_file, os.path.join(dialogues_dir, companion_name, 'train.txt'))
save_to_txt(val_file, os.path.join(dialogues_dir, companion_name, 'valid.txt'))
save_to_txt(test_file, os.path.join(dialogues_dir, companion_name, 'test.txt'))